In [19]:
import pandas as pd
import pycountry
from sqlalchemy import create_engine
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()


## World CO2 emissions from consumption of energy from theguardian.com

In [51]:
co2 = pd.read_excel('World CO2 emissions from consumption of energy.xlsx',sheet_name= 'Total Carbon Dioxide Emissions')
co2.columns = co2.iloc[0,]
co2.reset_index(inplace= True)
co2 = co2.reindex(co2.index.drop(0))
co2.rename(columns = {'level_0':'rank_2009','level_1':'rank_2008','Rank, 2006':'rank_2006',
                     'ISO country code': 'iso_code'},inplace = True)
co2.columns.name = None
clean = co2.loc[~co2['iso_code'].isnull(),]


In [102]:
# Table!! Create the ISO Code and Country List
iso_country = clean[['iso_code','Country']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'Country':'country','index':'id'})
iso_country.head()

# remove duplicated isocode and year combinations
iso_country = iso_country.loc[~iso_country.duplicated(subset=['iso_code','country'])]

# UM map to both 'Wake Island' and U.S. Pacific Islands. After research, update UM for U.S. Pacific Islands to UM1
#iso_country[iso_country['country'] == 'U.S. Pacific Islands']['iso_code'] = 'UM1'
print(iso_country.head())
#iso_country[iso_country["iso_code"] == "CN"]["country"]="chinaa"


# manually add Micronesia, Federated States of to ISO mapping table after checking with NFA dataset
iso_country = iso_country.append(pd.DataFrame([[iso_country.shape[0],'FM','Micronesia, Federated States of'],
                                              [iso_country.shape[0]+1,'NA','Namibia']],
                                columns = iso_country.columns),ignore_index = True)

iso_country


   id iso_code        country
0   0       CN          China
1   1       US  United States
2   2       IN          India
3   3       RU         Russia
4   4       JP          Japan


,id,iso_code,country
0,0,CN,China
1,1,US,United States
2,2,IN,India
3,3,RU,Russia
4,4,JP,Japan
5,5,DE,Germany
6,6,CA,Canada
7,7,KR,"Korea, South"
8,8,IR,Iran
9,9,GB,United Kingdom


In [53]:
rank_list = ['rank_2009','rank_2008','rank_2006','% change,  2008 to 2009','% change 2000 to 2009','Change in place, 2008 to 2009']
clean_rank_change = clean[[*rank_list,'iso_code']].set_index('iso_code')
clean_rank_change.head()
#remove the percentage changes
clean_rank_new = clean_rank_change[['rank_2009','rank_2008','rank_2006']].rename(columns = lambda x : x.replace('rank_',''))

clean_rank_new = clean_rank_new.stack().reset_index().rename(columns = {'level_1':'year',0:'rank'})

##Table!! iso_code-year-rank
clean_rank_new = clean_rank_new[~clean_rank_new.duplicated(subset = ['iso_code','year'])]
clean_rank_new.head()

,iso_code,year,rank
0,CN,2009,1
1,CN,2008,1
2,CN,2006,2
3,US,2009,2
4,US,2008,2


In [54]:
##Table!! Create the Country, Year, and CO2 Emission table
clean_co2 = clean.drop(rank_list,axis = 1)
clean_co2 = clean_co2.set_index('iso_code').drop('Country',axis = 1)
clean_co2 = clean_co2.stack().reset_index().rename(columns = {'level_1':'year',0:'co2_emission'})

# remove duplicated isocode and year combinations
clean_co2 = clean_co2.loc[~clean_co2.duplicated(subset=['iso_code','year'])]

## National Footprint Accounts 2018 from kaggle.com

In [141]:
NFA_df1=pd.read_csv("NFA_2018.csv")
co2 = pd.read_excel('World CO2 emissions from consumption of energy.xlsx',sheet_name= 'Total Carbon Dioxide Emissions')
print(NFA_df1.shape)
print(NFA_df1.columns)
NFA_df1.head()
NFA_df1=NFA_df1.rename(columns={'ISO alpha-3 code':'alpha_3'})

(87020, 15)
Index(['country', 'ISO alpha-3 code', 'UN_region', 'UN_subregion', 'year',
       'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon', 'total', 'Percapita GDP (2010 USD)',
       'population'],
      dtype='object')


# REYNA ADDED THIS PART TO UPDATE COUNTRY AND ISO MAPPING

In [142]:
#find not joinable countries for NFA and CO2 datasets
merged_nfa = pd.merge(NFA_df1,iso_country,on = 'country',how = 'left')
unique_country_nfa = sorted(merged_nfa[merged_nfa.iso_code.isnull()]['country'].unique())
merged_co2 = pd.merge(NFA_df1,iso_country,on = 'country',how = 'right')
unique_country_co2 = sorted(merged_co2[merged_co2.alpha_3.isnull()]['country'].unique())

#manually create the mapping for not matching countries in NFA. Will use this mapping to update NFA country names

to_change = {'Brunei Darussalam':'Brunei',
 'Cabo Verde':'Cape Verde',
 'Congo, Democratic Republic of':'Congo, Dem Rep',
 'Czechoslovakia':'',#no longer exists
 "Côte d'Ivoire":'Cote dIvoire (IvoryCoast)',
 'Ethiopia PDR':'', #no longer exists
 'Iran, Islamic Republic of':'Iran',
 "Korea, Democratic People's Republic of":'Korea, North',
 'Korea, Republic of':'Korea, South',
 "Lao People's Democratic Republic":'Laos',
 'Libyan Arab Jamahiriya':'Libya',
 'Macedonia TFYR':'Macedonia',
 #'Micronesia, Federated States of':'', #added in map_iso_country
 'Myanmar':'Burma',
# 'Namibia', #added in map_iso_country
 'Russian Federation':'Russia',
 'Réunion':'Reunion',
 'Serbia and Montenegro':'', #no longer exists
 'South Sudan':'Sudan',
 'Sudan (former)':'', #no longer exists
 'Syrian Arab Republic':'Syria',
 'Tanzania, United Republic of':'Tanzania',
 'Timor-Leste':'Timor-Leste (East Timor)',
 'USSR':'', #Soviet Union no longer exists
 'United States of America':'United States',
 'Venezuela, Bolivarian Republic of':'Venezuela',
 'Viet Nam':'Vietnam',
 'World':'', #world is not a country
 'Yugoslav SFR':'' #no longer exists
            }

#update NFA not matching country names
NFA_df1['country'].replace(to_change,inplace = True)
NFA_df = pd.merge(NFA_df1,iso_country,on = 'country',how = 'left')
#remove not valid countries that was set '' in previously in dictionary
NFA_df = NFA_df[NFA_df['country'] != '']

#sanity check to see if there are still not matched ISO code
NFA_df[NFA_df.iso_code.isnull()]

,country,alpha_3,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population,id,iso_code


In [106]:
# create country/ountry_code table and clean it
countries_df=NFA_df[["alpha_3","country"]].drop_duplicates(keep="first")
print(countries_df.shape)
# find missing values in  column
print(len(countries_df["alpha_3"].unique()) == len(countries_df["country"].unique()))

drop_list=countries_df[countries_df["alpha_3"].isna()]["country"]
print(drop_list)
countries_df=countries_df[~(countries_df["country"].isin(drop_list))]

(194, 2)
False
23020    
Name: country, dtype: object


In [107]:
countries_df

,alpha_3,country
0,ARM,Armenia
230,AFG,Afghanistan
770,ALB,Albania
1310,DZA,Algeria
1850,AGO,Angola
2390,ATG,Antigua and Barbuda
2740,ARG,Argentina
3280,AUS,Australia
3820,AUT,Austria
4360,BHS,Bahamas


In [22]:
NFA_2_df=NFA_df[~(NFA_df["country"].isin(drop_list))]

In [23]:
regions_df=NFA_2_df[['country','UN_region','UN_subregion']].drop_duplicates(keep="first").reset_index(drop=True)

In [24]:
population_df=NFA_2_df[["country","year","population"]].drop_duplicates(keep="first").reset_index(drop=True)

In [25]:
records_list_df=NFA_df["record"].drop_duplicates(keep="first")

In [26]:
e_footprint_df=NFA_2_df[["country","year",'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon']].loc[NFA_df["record"].isin(["BiocapTotGHA"," EFConsTotGHA","EFExportsTotGHA","EFImportsTotGHA","EFProdTotGHA"])]

In [27]:
df_names=[countries_df,regions_df,population_df,e_footprint_df]
table_names=["countries","regions","population","e_footprint"]

## Use pandas to load csv converted DataFrame into database

In [44]:
rds_connection_string = "root:mohammad@localhost/national_footprint"
engine = create_engine(f'mysql://{rds_connection_string}')

In [29]:
for i in range(len(df_names)):
    df_names[i].to_sql(name =table_names[i],if_exists = 'replace', con = engine, index = False)

### Confirm data has been added by querying the customer_location table
pd.read_sql_query('select * from regions', con=engine).head()

NameError: name 'engine' is not defined

## countries' land area from wikipedia page

In [30]:
url="https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area"
country_area_df=pd.read_html(url)[0]
print(country_area_df.head())
# clean the country_area_df 
columns=list(country_area_df.loc[0])
#print(columns)
columns=['Rank', 'sovereign state/dependency', 'Total_in_km2(mi2)', 'Land_in_km2(mi2)', 'Water_in_km2(mi2)', '%water', 'Notes']
country_area_df.columns=columns
country_area_df=country_area_df.iloc[2:]
country_area_df=country_area_df.drop(columns="Rank").reset_index(drop=True).sort_values("sovereign state/dependency")
print(country_area_df.shape)
country_area_df.head()

      0                           1                   2                  3  \
0  Rank  Sovereign state/dependency  Total in km2 (mi2)  Land in km2 (mi2)   
1   NaN                       World       (196,940,000)       (57,510,000)   
2   NaN                      Russia         (6,601,670)        (6,323,482)   
3   NaN                  Antarctica         (5,400,000)        (5,400,000)   
4   NaN                      Canada         (3,855,100)        (3,511,023)   

                    4        5  \
0  Water in km2 (mi2)  % water   
1       (139,434,000)     70.8   
2           (278,200)     4.21   
3                 (0)        0   
4           (344,080)     8.93   

                                                   6  
0                                              Notes  
1                                                NaN  
2  Largest country in the world (10.995% of the w...  
3  13,720,000 km2 (5,300,000 sq mi) (98%) of land...  
4  Largest English and French-speaking country an..

,sovereign state/dependency,Total_in_km2(mi2),Land_in_km2(mi2),Water_in_km2(mi2),%water,Notes
174,Abkhazia,"(3,340)",NaN,NaN,NaN,[Note 49]
42,Afghanistan,"(251,830)","(251,830)",(0),0,NaN
224,Akrotiri and Dhekelia (United Kingdom),(98.0),not determined,not determined,not determined,There is a salt lake and some wetlands of unkn...
146,Albania,"(11,100)","(10,578)",(520),4.7,NaN
10,Algeria,"(919,595)","(919,595)",(0),0,Largest Arabic-speaking and Berber-speaking co...


In [31]:
trial=pd.merge(iso_country,countries_df,how="outer",on="country").sort_values("country").reset_index(drop=True)
trial4=trial3[trial3.isnull().any(axis=1)]
print(trial4)

NameError: name 'iso_country' is not defined

In [39]:
trial3=pd.merge(iso_country,countries_df,how="outer",on="country").sort_values("country").reset_index(drop=True)
trial4=trial3[trial3.isnull().any(axis=1)]
print(trial4)

        id iso_code                                 country alpha_3
3    180.0       AS                          American Samoa     NaN
5    191.0       AQ                              Antarctica     NaN
27   105.0       BN                                  Brunei     NaN
28     NaN      NaN                       Brunei Darussalam     BRN
31    92.0       MM                                   Burma     NaN
33     NaN      NaN                              Cabo Verde     CPV
37   188.0       CV                              Cape Verde     NaN
46   143.0       CD                          Congo, Dem Rep     NaN
47     NaN      NaN           Congo, Democratic Republic of     COD
50   114.0       CI               Cote dIvoire (IvoryCoast)     NaN
55     NaN      NaN                          Czechoslovakia    CSHH
56     NaN      NaN                           Côte d'Ivoire     CIV
68   212.0       FK       Falkland Islands (Islas Malvinas)     NaN
69   173.0       FO                           Fa

In [45]:
len(list(pycountry.countries))

249